In [2]:
%%writefile d:\myPyCode\팀프로젝트\netflix_Unofficial_API_module.py

import requests
import pandas as pd
# from bs4 import BeautifulSoup
# import urllib.request
import json

url = "https://netflix-unofficial.p.rapidapi.com/api/"

select_netflix_mode = ['genres','search','countries']
# genres,countries = 항목별 뽑아야한다.
# search = 분류로 나눠야 한다.

headers = {
    'x-rapidapi-key': "c05cbb963bmshc6a47b50ae8c81cp167c7bjsn5f651e0b6361",
    'x-rapidapi-host': "netflix-unofficial.p.rapidapi.com"
    }

netflix_data = requests.request("GET", url+select_netflix_mode[1], headers=headers).json()

def netflix_API_request() :
    netflix_API_data = pd.DataFrame(requests.request("GET", url+select_netflix_mode[1], headers=headers))
    # netflix_API_data = netflix_API_data.fillna(method='ffill')
    # return netflix_API_data
    print(netflix_API_data.read().decode('utf-8'))

def netflix_API_init():
    netflix_keys_list = ['title']
    
    for a in json.loads(netflix_data[""]).keys() :
        netflix_keys_list.append(a)
        
    f = open("netflix_unofficial.csv","w",-1,"utf-8")
    for i in netflix_keys_list : 
        f.write(i+',')
    f.write('\n')

    for data_title in netflix_data.keys() :
        f.write(data_title.replace(','," ")+',')
        data_get_key = json.loads(netflix_data[data_title])
        for get_key in netflix_keys_list[1:] :
            f.write(str(data_get_key[get_key]).replace(',',' ')+',')

            # print(data_get_key[get_key])
        f.write('\n')

    f.close()





Overwriting d:\myPyCode\팀프로젝트\netflix_Unofficial_API_module.py


In [4]:
%%writefile d:\myPyCode\팀프로젝트\netflix_streaming_API_module.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import csv
from matplotlib import font_manager, rc

df = pd.read_csv("All_Streaming_Shows.csv",encoding="latin1")
select = pd.DataFrame(df)
select = select.fillna(method='ffill')

def netflix_API_data_init():
    df = pd.read_csv("All_Streaming_Shows.csv",encoding="latin1")
    select = pd.DataFrame(df)
    select = select.fillna(method='ffill')
    return select

# 날짜별순으로 값 원하는 값 뽑아내기 (type과 key값 )
def netflix_API_data_selection(select_col,Name,wants_type_standard) :
    #select.sort_values(by = ['release_year','date_added']) 날짜별로 구분하기
    selection = select[select[select_col] == Name].sort_values(by = wants_type_standard)
    #select[select[select_col] == Name] Name과 같은 select key값을 뽑아낸다.
    return selection

# 날짜순으로 타입안에 원하는 문자포함되어 있는 행 뽑아내기
def netflix_API_data_findword(netflix_framedata,position,Name,wants_type_standard) :
    find_line = netflix_framedata.loc[netflix_framedata[position].str.contains(Name)].sort_values(by= 
    wants_type_standard)
    return find_line

# 중복 없이 타입 뽑아내기
def API_remove_overlap(netflix_data,netflix_want_name):
    list_module = []
    netflix_data_find = set(list(netflix_data[netflix_want_name]))
    for i in netflix_data_find :
        list_module.append(i)
    return list_module

# 데이터프레임에서 반복 갯수세기
def API_netflix_count(netflix_dataframe,collect_type) :
    collect_lists = netflix_dataframe[collect_type].value_counts()
    return collect_lists



def grape_step5(select,years_listed_in,Genre_lists,streamings):
    streaming_data = pd.DataFrame()
    # dataframe 초기 설정
    streaming_data = streaming_data.reindex(index=streamings)
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
    for years_list in years_listed_in:
        New_select = netflix_API_data_selection('Year Released',years_list,"Streaming Platform")
        streaming_data[Genre_lists] = 0
    # New_select = netflix_API_data_selection('Year Released',2020,"Streaming Platform")
        # plt.figure(figsize=(7,7))
        for Genre_list in Genre_lists :
            Genres_count= {}
            #원하는장르포함 모든 행 뽑기
            listed_in_pullout = netflix_API_data_findword(New_select,'Genre', Genre_list, 'Year Released')
            # 시대별 기준으로 나누기
            listed_in_count = API_netflix_count(listed_in_pullout,'Streaming Platform') 
            # 아이템으로 정렬하면 key값을 정렬가능
            listed_in_count = dict(sorted(listed_in_count.items(),reverse=True))
            
            for streaming in streamings :
                Genre_countcount=0
                for list_in_count in listed_in_count.keys() :
                    if streaming in list_in_count :
                        Genre_countcount = Genre_countcount + int(listed_in_count[list_in_count])
                    Genres_count[streaming]=Genre_countcount
                    streaming_data.loc[streaming,[Genre_list]]=Genres_count[streaming]
            
            
            # streaming_data.plot(kind='pie',y=Genre_list,ax=plt.subplot(3,3,num), shadow = True,rotatelabels = False, startangle=90, autopct='%1.1f%%',fontsize=5)
        ax = streaming_data.plot(kind='bar',stacked=True,fontsize=5)

        # plt.ylabel(Genre_list,rotation=0,fontsize=8,loc='top')
        plt.legend(loc='best',fontsize=5,shadow = True)
        plt.suptitle(str(years_list)+'년 장르별 스트리밍 비교')
        #https://kongdols-room.tistory.com/86

Overwriting d:\myPyCode\팀프로젝트\netflix_streaming_API_module.py


In [1]:
%%writefile d:\myPyCode\팀프로젝트\netflix_frame_module.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import csv
import seaborn as sns
import json

df = pd.read_csv("netflix_titles.csv",encoding="latin1")
select = pd.DataFrame(df,columns = ['show_id', 'type', 'title','country', 'date_added','release_year', 'rating',
'duration','listed_in', 'description'])

# 빈 값 처리
select = select.fillna(method='ffill')

# 초기 데이터프레임 뽑아내기
def netflix_data_init() :
    df = pd.read_csv("netflix_titles.csv",encoding="latin1")
    select = pd.DataFrame(df,columns = ['show_id', 'type', 'title','country', 'date_added','release_year',
    'rating','duration','listed_in', 'description'])
    return select

# 날짜별순으로 값 원하는 값 뽑아내기 (type과 key값 )
def netflix_date_selection(select_col,Name) :
    #select.sort_values(by = ['release_year','date_added']) 날짜별로 구분하기
    selection = select[select[select_col] == Name].sort_values(by = ['release_year','date_added'])
    #select[select[select_col] == Name] Name과 같은 select key값을 뽑아낸다.
    return selection

# 날짜순으로 타입안에 원하는 문자포함되어 있는 행 뽑아내기
def netflix_data_findword(netflix_framedata,position,Name) :
    find_line = netflix_framedata.loc[netflix_framedata[position].str.contains(Name)].sort_values(by= 
    ['release_year','date_added'])
    return find_line

# 중복 없이 타입 뽑아내기
def remove_overlap(netflix_data,netflix_want_name):
    list_module = []
    netflix_data_find = set(list(netflix_data[netflix_want_name]))
    for i in netflix_data_find :
        list_module.append(i)
    return list_module

# 데이터프레임에서 반복 갯수세기
def netflix_count(netflix_dataframe,collect_type) :
    collect_lists = netflix_dataframe[collect_type].value_counts()
    return collect_lists
    
# 원 그래프 만들기
# def circle_grape(label_X,label_Y):
#     plt.pie(label_X)
#     plt.pie(label_X, labels=label_Y, autopct='%0.1f%%')
#     plt.show()


# 선형그래프 뽑아내기
def analyze_netflix_year(data_X,data_Y,data_title):
    p1 = plt.plot(data_X,data_Y,'r^--',label=data_title)
    plt.legend()
    plt.show()

# 산점도 그래프 그리기
def circle_3D_grape(X,Y,Z):
    ax = plt.figure().add_subplot(111,projection='3d')
    ax.scatter(X,Y,Z)

    ax.set_xlabel('year')
    ax.set_ylabel('type')
    ax.set_zlabel('age')
    plt.suptitle('aaaa',fontsize=16)
    plt.show()


# 데이터 정렬하기 1) 영화장르
def netflix_data_genre_years_clear(movie_listed_in,years_listed_in, netflix_data):

    # 시대별 영화장르 뽑아내기
    df = pd.DataFrame( )
    df = df.reindex(index = years_listed_in)

    for movie_elements in movie_listed_in :
    
        #원하는장르포함 모든 행 뽑기
        listed_in_pullout = netflix_data_findword(netflix_data,'listed_in', movie_elements)
        # 시대별 기준으로 나누기
        listed_in_count = netflix_count(listed_in_pullout,'release_year') 
        # 아이템으로 정렬하면 key값을 정렬가능
        listed_in_count = dict(sorted(listed_in_count.items(),reverse=True))
        # dataframe 초기 설정
        df = df.reindex(index=years_listed_in)
        df[movie_elements] = 0
    
        for index_elements in years_listed_in :
            if index_elements in list(listed_in_count.keys()) :
                df.loc[index_elements,[movie_elements]]=listed_in_count[index_elements]

    return df

# 데이터 정렬하기 1) 나이 장르
def netflix_data_rating_years_clear(age_listed_in,years_listed_in, netflix_data):

    # 시대별 영화장르 뽑아내기
    df = pd.DataFrame( )
    df = df.reindex(index = years_listed_in)

    for age_elements in age_listed_in :
    
        #원하는장르포함 모든 행 뽑기
        listed_in_pullout = netflix_data_findword(netflix_data,'rating', age_elements)
        # 시대별 기준으로 나누기
        listed_in_count = netflix_count(listed_in_pullout,'release_year') 
        # 아이템으로 정렬하면 key값을 정렬가능
        listed_in_count = dict(sorted(listed_in_count.items(),reverse=True))
        # dataframe 초기 설정
        df = df.reindex(index=years_listed_in)
        df[age_elements] = 0
    
        for index_elements in years_listed_in :
            if index_elements in list(listed_in_count.keys()) :
                df.loc[index_elements,[age_elements]]=listed_in_count[index_elements]

    return df

# 나라를 기준으로 데이터 추출
def grape_step1(netflix_data,movie_listed_in,years_country_listed_in,country_listed_in,colors):
    a=1
    fig = plt.figure(figsize=(9,9)) 
    # 나라별 데이터 뽑기
    Num=1
    for netflix_country_lists in country_listed_in :
        netflix_country_data =  netflix_data_findword(netflix_data,'country',netflix_country_lists)
        netflix_country_dataFrame = netflix_data_genre_years_clear(movie_listed_in,years_country_listed_in, 
        netflix_country_data)
        # 막대그래프화
        netflix_country_dataFrame.plot(kind='bar',title= netflix_country_lists,color=colors,ax=fig.add_subplot(1,3,a))
        a=a+1
        plt.show()


def grape_step3(netflix_data,years_age_listed_in,country_listed_in,colors):
    a=1
    fig = plt.figure(figsize=(9,9)) 
    # 나이별 데이터 뽑기
    for netflix_country_lists in country_listed_in :
        netflix_country_data =  netflix_data_findword(netflix_data,'country',netflix_country_lists)
        # 연령 시청 종류 조사
        netflix_age_list = remove_overlap(netflix_country_data,'rating')
        # 연령별 데이터 뽑기
        netflix_age_dataFrame = netflix_data_rating_years_clear(netflix_age_list, years_age_listed_in,
        netflix_country_data)
        # 막대그래프화
        netflix_age_dataFrame.plot(kind='bar',title= netflix_country_lists,color=colors,ax=fig.add_subplot(1,3,a))
        a=a+1
        plt.show()


# step 4 나라 하나 당 연도별 그래프 뽑기 
# 그래프 속 내용 : 장르별, 나이별 :  count

def grape_step4_pre(year,nation,netflix_age,movie_listed_in):
    # 년도별 뽑는다.
    select_years = netflix_date_selection('release_year',year)
    # 년도별로 뽑은걸 나라별로 뽑는다.
    select_year = select_years[select_years['country'] == nation].sort_values(by = ['release_year','date_added'])
    
    # 비어있는 데이터프레임 생성
    df = pd.DataFrame()
    # 행값의 네임 설정 (netflix_age)
    df = df.reindex(index=netflix_age)
    
    #데이터 프레임 설정 장르별,나이별 count
    for a in netflix_age :
        find_age_year = netflix_data_findword(select_year,'rating',a)
        for i in movie_listed_in :
            select_movie_year = netflix_data_findword(find_age_year,'listed_in',i)
            select_age_movie_year = dict(netflix_count(select_movie_year,'listed_in'))
            df.loc[a,[i]]=len(select_age_movie_year)

    sns.heatmap(df,annot=True,fmt='0.0f',annot_kws={"size": 5}, cmap='cubehelix')
    plt.title(str(year)+' - '+nation,fontsize=5)
    plt.show()
#https://analytics4everything.tistory.com/66 참고 

def grape_step4(country_listed_in,movie_listed_in):
    netflix_age = remove_overlap(select,'rating')
    a=1
    sns.set(font_scale=0.5)
    fig = plt.figure(figsize=(9,9))
    
    for country_list in country_listed_in:
        for year_num in range(2017,2020):
            fig.add_subplot(3,3,a)
            grape_step4_pre(year_num,country_list,netflix_age,movie_listed_in)
            a=a+1

# 행 열 변환 팁
# listed_in_count.keys()
# a = pd.DataFrame(comedy_count,index=['장르'])
# a = a.transpose()
# a

    



Overwriting d:\myPyCode\팀프로젝트\netflix_frame_module.py


In [2]:
cd d:\myPyCode\팀프로젝트

d:\myPyCode\팀프로젝트


In [3]:
%matplotlib tk

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from netflix_frame_module import *
import seaborn as sns
# import json
# from netflix_Unofficial_API_module import *
from netflix_streaming_API_module import *

movie_listed_in = ['Comed','Horror','Independ','Sci',
                   'Thrille','Drama','Classic','Adventure',
                  'Family','Romantic','Music','Classic',
                  'International']

color = sns.color_palette('Dark2')
#https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html 참조

years_country_listed_in = list(range(2010,2020))
years_age_listed_in = list(range(2016,2021))
country_listed_in = ['United States','South Korea','United Kingdom']

API_years_listed_in = range(2017,2020)
streamings = ['Netflix','Free Services','Prime Video','Hulu','fuboTV','Funimation']
API_Genre_lists =  ['Comed','Horror','Animation','Sci','Thrille','Drama','Adventure','Family','Romance']

API_select = netflix_API_data_init()


# 초기 데이터
netflix_data = netflix_data_init()
# netflix_API_init()
# 데이터 프레임 null 값 처리하기
netflix_data = netflix_data.fillna(method='ffill')
# a=netflix_API_request()


grape_step1(netflix_data,movie_listed_in,years_country_listed_in,country_listed_in,color)
grape_step3(netflix_data,years_age_listed_in,country_listed_in,color)
grape_step4(country_listed_in,movie_listed_in)
grape_step5(API_select,API_years_listed_in,API_Genre_lists,streamings)



In [5]:
%%writefile d:\myPyCode\팀프로젝트\all_streaming_비교.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import csv
import seaborn as sns
import json

df = pd.read_csv("All_Streaming_Shows.csv",encoding="latin1")
select = pd.DataFrame(df)
select

Writing d:\myPyCode\팀프로젝트\all_streaming_비교.py


In [ ]:
# comedy_pullout = netflix_data_findword('listed_in','Comed')
# comedy_count = netflix_count(comedy_pullout,'release_year')
# comedy_lists = netflix_count(comedy_pullout,'release_year').keys()
# comedy_count = dict(sorted(comedy_count.items(),reverse=True))
# analyze_netflix_year(list(comedy_count.keys()),list(comedy_count.values()),'comedy')
# analyze_netflix_year


In [5]:
cd d:\myPyCode\팀프로젝트

d:\myPyCode\팀프로젝트


In [9]:

%matplotlib tk

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from netflix_frame_module import *
from netflix_Unofficial_API_module import netflix_init

# comedy_pullout = netflix_data_findword(netflix_data_init(),'listed_in','Comed')
# comedy_count = netflix_count(comedy_pullout,'release_year')
# comedy_lists = netflix_count(comedy_pullout,'release_year').keys()
# comedy_count = pd.DataFrame(comedy_count)
# comedy_count
colors = ['black','dimgray','dimgrey','darkgray','silver','lightgrey']
a =['title','country','release_year', 'rating']
comedy_count = netflix_data_init()
comedy_count = comedy_count.head()
comedy_count
# for i,j in zip (a,range(len(a))) :
#     b = comedy_count[comedy_count[i]==j].index
#     comedy_count.loc[b,'color'] = 'r'
# a=0.3

comedy_count.plot(kind='bar',color=colors)

#  netflix_age_dataFrame.plot(kind='bar', title= netflix_country_lists)#ax=fig.add_subplot(1,3,a)
#         a=a+1
#         plt.show()


<AxesSubplot:>

In [14]:
a = pd.DataFrame(comedy_count,index=['장르'])
a = a.transpose()
a
# a.columns.names=['연도별']
plt.subplots(1,1)
a.plot(kind='bar')
plt.show()


NameError: name 'comedy_count' is not defined

In [16]:
list_module = []

a = netflix_data_findword(netflix_data,'type','Movie')
b = set(list(a['listed_in']))
for i in b :
    list_module.append(i)
list_module


NameError: name 'netflix_data' is not defined

In [17]:

# list_module = []
# a = netflix_data_findword(netflix_data,'listed_in','Comed')
# b = set(list(a['rating']))
# for i in b :
#     list_module.append(i)
# list_module